# Clasificación de textos

* Francisco Josse Roja Rojas (frrojasr@unal.edu.co)
* Yeira Liseth Rodríguez Rodríguez (yrodriguezro@unal.edu.co)
<br>
<br>
A continuación se presenta el desarrollo del tercer ejercicio del capítulo 5 de redes neuronales. Para ello se uso el dataset Reuters de TensorFlow, que contiene 11,228 noticias clasificadas en 46 categorías temáticas

#### 1. Cargar y explorar el dataset
El conjunto de datos Reuters contiene textos ya tokenizados (cada palabra ha sido transformada en un número entero según su frecuencia). Aquí se usa solo las 10,000 palabras más frecuentes para evitar ruido y facilitar el entrenamiento.

In [1]:
from tensorflow.keras.datasets import reuters
from tensorflow.keras.preprocessing.sequence import pad_sequences
from tensorflow.keras.utils import to_categorical

# Cargar el dataset
(x_train, y_train), (x_test, y_test) = reuters.load_data(num_words=10000)

# Mostrar tamaño de los conjuntos
print(f"Número de muestras de entrenamiento: {len(x_train)}")
print(f"Número de muestras de prueba: {len(x_test)}")
print(f"Ejemplo de entrada (tokenizada): {x_train[0]}")
print(f"Etiqueta asociada: {y_train[0]}")


2110848/2110848 ━━━━━━━━━━━━━━━━━━━━ 0s 0us/step
Número de muestras de entrenamiento: 8982
Número de muestras de prueba: 2246
Ejemplo de entrada (tokenizada): [1, 2, 2, 8, 43, 10, 447, 5, 25, 207, 270, 5, 3095, 111, 16, 369, 186, 90, 67, 7, 89, 5, 19, 102, 6, 19, 124, 15, 90, 67, 84, 22, 482, 26, 7, 48, 4, 49, 8, 864, 39, 209, 154, 6, 151, 6, 83, 11, 15, 22, 155, 11, 15, 7, 48, 9, 4579, 1005, 504, 6, 258, 6, 272, 11, 15, 22, 134, 44, 11, 15, 16, 8, 197, 1245, 90, 67, 52, 29, 209, 30, 32, 132, 6, 109, 15, 17, 12]
Etiqueta asociada: 3


#### 2. Procesamiento
Los textos tienen longitudes variables, por lo que usamos padding para que todos tengan 200 palabras. Esto es necesario para que la red pueda procesarlos como vectores de igual tamaño. Las etiquetas se convierten a categorías one-hot para usar `categorical_crossentropy`.

In [2]:
# Padding: igualar longitud de todas las secuencias
maxlen = 200
x_train_pad = pad_sequences(x_train, maxlen=maxlen)
x_test_pad = pad_sequences(x_test, maxlen=maxlen)

# One-hot encoding de las etiquetas (46 clases)
num_classes = max(y_train) + 1
y_train_cat = to_categorical(y_train, num_classes)
y_test_cat = to_categorical(y_test, num_classes)

#### 3. Construcción de la red neuronal
* `Embedding`: convierte cada palabra (número) en un vector de tamaño 64, permitiendo al modelo aprender representaciones semánticas.

* `Flatten`: aplanamos para alimentar a una capa densa.

* `Dense + ReLU`: capa oculta de 128 neuronas.

* `Dropout`: ayuda a reducir overfitting.

* `Dense + softmax`: capa final para clasificar entre 46 temas.

In [7]:
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Embedding, Flatten, Dense, Dropout

model = Sequential([
    Embedding(input_dim=10000, output_dim=64, input_shape=(maxlen,)),  # entrada explícita
    Flatten(),
    Dense(128, activation='relu'),
    Dense(64, activation='relu'),           # capa adicional
    Dropout(0.3),                            # dropout más conservador
    Dense(46, activation='softmax')         # salida multicategoría
])

# Construir manualmente para que summary funcione correctamente
model.build(input_shape=(None, maxlen))
model.summary()



/usr/local/lib/python3.11/dist-packages/keras/src/layers/core/embedding.py:93: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(**kwargs)


Model: "sequential_1"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ embedding_1 (Embedding)         │ (None, 200, 64)        │       640,000 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ flatten_1 (Flatten)             │ (None, 12800)          │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_2 (Dense)                 │ (None, 128)            │     1,638,528 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_3 (Dense)                 │ (None, 64)             │         8,256 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dropout_1 (Dropout)             │ (None, 64)             │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_4 (Dense)                 │ (None, 46)             │         2,990 │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 2,289,774 (8.73 MB)

 Trainable params: 2,289,774 (8.73 MB)

 Non-trainable params: 0 (0.00 B)

#### 4. Entrenamiento del modelo
Se entrena el modelo usando el 80% del conjunto de entrenamiento y reservamos el 20% para validación. Esto permite monitorear si el modelo está sobreajustando (overfitting).



In [8]:
model.compile(optimizer='adam',
              loss='categorical_crossentropy',
              metrics=['accuracy'])

history = model.fit(x_train_pad, y_train_cat,
                    epochs=20,                     # más épocas
                    batch_size=128,
                    validation_split=0.2,
                    verbose=1)

Epoch 1/20
57/57 ━━━━━━━━━━━━━━━━━━━━ 7s 71ms/step - accuracy: 0.3009 - loss: 2.9512 - val_accuracy: 0.5181 - val_loss: 1.8216
Epoch 2/20
57/57 ━━━━━━━━━━━━━━━━━━━━ 5s 68ms/step - accuracy: 0.5551 - loss: 1.7763 - val_accuracy: 0.6049 - val_loss: 1.5478
Epoch 3/20
57/57 ━━━━━━━━━━━━━━━━━━━━ 6s 80ms/step - accuracy: 0.6762 - loss: 1.3028 - val_accuracy: 0.6578 - val_loss: 1.3927
Epoch 4/20
57/57 ━━━━━━━━━━━━━━━━━━━━ 4s 62ms/step - accuracy: 0.7845 - loss: 0.9012 - val_accuracy: 0.6772 - val_loss: 1.3617
Epoch 5/20
57/57 ━━━━━━━━━━━━━━━━━━━━ 7s 93ms/step - accuracy: 0.8629 - loss: 0.5936 - val_accuracy: 0.6728 - val_loss: 1.4530
Epoch 6/20
57/57 ━━━━━━━━━━━━━━━━━━━━ 9s 74ms/step - accuracy: 0.8997 - loss: 0.4251 - val_accuracy: 0.6800 - val_loss: 1.4855
Epoch 7/20
57/57 ━━━━━━━━━━━━━━━━━━━━ 5s 93ms/step - accuracy: 0.9308 - loss: 0.3105 - val_accuracy: 0.6778 - val_loss: 1.6069
Epoch 8/20
57/57 ━━━━━━━━━━━━━━━━━━━━ 4s 72ms/step - accuracy: 0.9461 - loss: 0.2302 - val_accuracy: 0.6811 - v

#### 5. Evaluación del modelo
La evaluación nos da una medida objetiva de rendimiento del modelo en datos nuevos. Una buena red debería superar el 75-80% de precisión con esta arquitectura.

In [9]:
loss, acc = model.evaluate(x_test_pad, y_test_cat, verbose=0)
print(f"\nPrecisión en test: {acc * 100:.2f}%")



Precisión en test: 67.59%


#### 6. Pruebas con base en los pesos aprendidos
Se toma una muestra de entrada, se calcula manualmente la activación de la primera capa densa usando los pesos aprendidos y se comparan las predicciones del modelo con la etiqueta real.

In [11]:
import numpy as np
from tensorflow.keras.datasets import reuters
from tensorflow.keras.preprocessing.sequence import pad_sequences
from tensorflow.keras.utils import to_categorical
import tensorflow as tf

# 1. Cargar el conjunto de datos
(x_train, y_train), (x_test, y_test) = reuters.load_data(num_words=10000)
word_index = reuters.get_word_index()
reverse_word_index = {value: key for key, value in word_index.items()}

# 2. Función para decodificar texto desde índices
def decode_text(encoded):
    return ' '.join([reverse_word_index.get(i - 3, '<?>') for i in encoded])

# 3. Mapeo aproximado de etiquetas a categorías
label_map = {
    0:  "cocoa", 1:  "grain", 2:  "crude oil", 3:  "money-fx", 4:  "interest rates",
    5:  "ship", 6:  "trade", 7:  "reserves", 8:  "cotton", 9:  "coffee",
    10: "sugar", 11: "gold", 12: "tin", 13: "strategic metals", 14: "money-supply",
    15: "income", 16: "GNP", 17: "inflation", 18: "bond", 19: "housing",
    20: "jobs", 21: "retail", 22: "money market", 23: "earnings", 24: "acquisitions",
    25: "merger", 26: "commodity", 27: "bankruptcy", 28: "investment", 29: "legal",
    30: "budget", 31: "defense", 32: "energy", 33: "strategic reserve", 34: "natural gas",
    35: "petrochemicals", 36: "opec", 37: "economy", 38: "finance", 39: "real estate",
    40: "automobile", 41: "transportation", 42: "food", 43: "textile", 44: "electronics", 45: "tourism"
}

# 4. Preprocesamiento (asegúrate que estos pasos ya se hicieron antes de entrenar el modelo)
maxlen = 200
x_test_pad = pad_sequences(x_test, maxlen=maxlen)

# 5. Seleccionar una muestra para prueba
sample_idx = 0
input_sample = x_test_pad[sample_idx:sample_idx+1]
original_encoded = x_test[sample_idx]
decoded_text = decode_text(original_encoded)
true_label = y_test[sample_idx]

# 6. Obtener activaciones manualmente desde el modelo
weights, biases = model.layers[2].get_weights()  # primera capa densa
embedding_output = model.layers[0](input_sample)
flattened = model.layers[1](embedding_output)
flattened_np = flattened.numpy()
z = np.dot(flattened_np, weights) + biases
a = np.maximum(0, z)  # ReLU activación

# 7. Predicción del modelo
y_pred = model.predict(input_sample, verbose=0)
predicted_label = np.argmax(y_pred)
top3_probs = np.round(np.sort(y_pred[0])[-3:][::-1], 3)
top3_classes = np.argsort(y_pred[0])[-3:][::-1]

# 8. Mostrar resultados visuales y explicativos
print("\n📰 Texto de la noticia (fragmento reconstruido):")
print("--------------------------------------------------")
print(decoded_text[:500], "...")
print("--------------------------------------------------")
print(f"🎯 Etiqueta real: {true_label} ({label_map[true_label]})")
print(f"🤖 Etiqueta predicha: {predicted_label} ({label_map[predicted_label]})")

print("\n🔥 Activaciones de la primera capa (primeras 5 neuronas):")
print(np.round(a[0][:5], 3))

print("\n📊 Top 3 clases más probables:")
for i in range(3):
    label = top3_classes[i]
    print(f"  Clase {label} ({label_map[label]}) - Probabilidad: {top3_probs[i]}")


📰 Texto de la noticia (fragmento reconstruido):
--------------------------------------------------
<?> the great atlantic and pacific tea co said its three year 345 mln dlr capital program will be be substantially increased to <?> growth and expansion plans for <?> inc and <?> inc over the next two years a and p said the acquisition of <?> in august 1986 and <?> in december helped us achieve better than expected results in the fourth quarter ended february 28 its net income from continuing operations jumped 52 6 pct to 20 7 mln dlrs or 55 cts a share in the latest quarter as sales increased 4 ...
--------------------------------------------------
🎯 Etiqueta real: 3 (money-fx)
🤖 Etiqueta predicha: 3 (money-fx)

🔥 Activaciones de la primera capa (primeras 5 neuronas):
[1.677 1.548 0.699 1.536 2.083]

📊 Top 3 clases más probables:
  Clase 3 (money-fx) - Probabilidad: 0.8169999718666077
  Clase 4 (interest rates) - Probabilidad: 0.17299999296665192
  Clase 20 (jobs) - Probabilidad: 0.0040